In [20]:
# 1. Install All the Required Packages

!pip install openai tiktoken chromadb langchain GitPython langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.4 MB/s eta 0:00:00


In [26]:
# 2. LangChain version

import langchain
import langchain_community

print(langchain.__version__)

print(langchain_community.__version__)

0.2.6
0.2.6


In [31]:
# 3. LangChain — Amazon Bedrock




In [14]:
#Step 02: Import All the Required Libraries

# To clone the github repo in the Jupyter Notebook
from git import Repo
from langchain.text_splitter import Language
# To load the files
from langchain.document_loaders.generic import GenericLoader

# In the Language Parser we will pass the extension of the programming language we are working with,
# Here we are working with Pyton programming language so we will add Python, if we are working with
#Java, C++ or any other programming language we can add that extension as well
from langchain.document_loaders.parsers import LanguageParser

# After loading all the code in a variable we will split the code into small chunks using Recursive Character Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import sys
# We will download OpenAI Embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
# We will save the embeddings in the Chroma Vector Store
from langchain.vectorstores import Chroma
# We will create a ChatOpenAI wrapper, to chat with our document
from langchain.chat_models import ChatOpenAI
# We can add memory to our Q/A chain using Conversation Summary Memory, it will look at the historical conversation
# Computer the Summary and add that to a subsequent conversation
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

ModuleNotFoundError: Module langchain_community.document_loaders.generic not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`